In [31]:
import secrets
import string
import pandas as pd
import hashlib
import base64
import os

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes



In [32]:
csv_path = "../modified_data/hotel_booking_token.csv"
csv_output_path = "../modified_data/hotel_booking_token_rsaemail.csv"
public_path = "public_key.pem"

In [33]:

def generate_keys():
    if os.path.exists(public_path):
      return
    # Generate private key
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    
    # Generate public key
    public_key = private_key.public_key()
    
    # Save the private key
    with open("private_key.pem", "wb") as f:
        f.write(private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.PKCS8,
            encryption_algorithm=serialization.NoEncryption()
        ))
    
    # Save the public key
    with open("public_key.pem", "wb") as f:
        f.write(public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ))
# Usage
generate_keys()


In [34]:

# Load public key
with open("public_key.pem", "rb") as f:
    public_key = serialization.load_pem_public_key(
        f.read(),
        backend=default_backend()
    )
# Load private key
with open("private_key.pem", "rb") as f:
    private_key = serialization.load_pem_private_key(
        f.read(),
        password=None,
        backend=default_backend()
    )

def encrypt_message(message):
    # Encrypt message
    encrypted = base64.b64encode(public_key.encrypt(
        message.encode(),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )).decode('utf-8')
    return encrypted

def decrypt_message(encrypted_message):
    # Decrypt message
    original_message = private_key.decrypt(
        base64.b64decode(encrypted_message),
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return original_message.decode()

encrypted_message = encrypt_message("Hello, this is a secret message!")
print("Encrypted:", encrypted_message)
decrypted_message = decrypt_message(encrypted_message)
print("Decrypted:", decrypted_message)

Encrypted: B7qo1KbofWkZwo0cF2wOxQg3seznrbZaThmMiVlXwFJIur5cXLuwu/QlfGgCkVhfuVP+LAqkX+VCkpPRd69NzzCbmgvByzNJ4AyrIoTWVPP5/jG7qo9s9rlXZHT41f8Lysw4TgXE4gmR4H0EzmZKezTASUSmjMhVCTuI9NSCtpuExu+CMg7DCGy3FIeoNfL7GQVx5SVwYnqmpp0ruhZJnr5KdG4AN0muRLNxntTdB04W7+YqM8I1Ty8RPpSJkJHVqnIJsrJoMxVLUwmNcshR1Y2aaTi86fASYgZ/5jRd1b/5k21BkzTtPAAghn2Q2ZoR+o+8RIjudpo1CyzUFhdOHg==
Decrypted: Hello, this is a secret message!


In [35]:
total = 0
file_path = csv_path
simple_data = pd.read_csv(file_path)

total=len(simple_data)


In [36]:
encrpyted_emails = []

simple_data['email'] = simple_data['email'].apply(lambda x: encrypt_message(x))
print(simple_data['email'].values[0])
print(decrypt_message(simple_data['email'].values[0]))

simple_data.to_csv(csv_output_path, index=False)

CNrQjb7mh0zByrLQwfn4ZOmO04whfPx/dwxF5RyVuTANqRc86lW/3uLIzCLVbfSBctPvm8g8jEr7mT2GdDQG1Vvaih8k6ps1Mt6BzQKQ7+3/4IVwtN5tzsOdtsEZmP9asSInYYmObhad8PG+HI61yelQc62UaEGvS8ecUY9DS4aNOwe5o2vVaEPkrKSGNno8+TRA4WUnUy7bxPf0rYORvRsXjs7u/xklZGzXHy238ltG0s/Mhu5JQpydgEy+gdhKsZuZvF9SETPM01IkMnG7cPrWrX9a9Evl0BHEAuexqK1l0Pso0LYP+6/EcJsITVKrKRhw5a65dkhI6iGZB6ZumQ==
Ernest.Barnes31@outlook.com
